## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

In [ ]:
# Import necessary libraries:
# - load_dotenv: Loads environment variables from a .env file (e.g., your OpenAI API key).
# - OpenAI: The official OpenAI client to interact with their API.
# - PdfReader: Used to read and extract text from PDF files.
# - gr: Gradio is a UI library to quickly build web interfaces for machine learning apps.

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
"""
This script reads a PDF file located at 'me/profile.pdf' and extracts all the text from each page.
The extracted text is concatenated into a single string variable named 'linkedin'.
This can be useful for feeding structured content (like a resume or profile) into an AI model or for further text processing.
"""
reader = PdfReader("me/profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [ ]:
"""
This script loads a PDF file named 'projects.pdf' from the 'me' directory
and extracts text from each page. The extracted text is combined into a single
string variable called 'projects', which can be used later for analysis,
summarization, or input into an AI model.
"""

reader = PdfReader("me/projects.pdf")
projects = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        projects += text

In [ ]:
# Print for sanity checks
"Print for sanity checks"

print(linkedin)
print(projects)

In [ ]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Cristina Rodriguez"

In [ ]:
"""
This code constructs a system prompt for an AI agent to role-play as a specific person (defined by `name`).
The prompt guides the AI to answer questions as if it were that person, using their career summary,
LinkedIn profile, and project information for context. The final prompt ensures that the AI stays
in character and responds professionally and helpfully to visitors on the user's website.
"""

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n\n\n## Projects:\n{projects}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
system_prompt

In [ ]:
"""
This function handles a chat interaction with the OpenAI API.

It takes the user's latest message and conversation history,
prepends a system prompt to define the AI's role and context,
and sends the full message list to the GPT-4o-mini model.

The function returns the AI's response text from the API's output.
"""

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
"""
This line launches a Gradio chat interface using the `chat` function to handle user input.

- `gr.ChatInterface(chat, type="messages")` creates a UI that supports message-style chat interactions.
- `launch(share=True)` starts the web app and generates a public shareable link so others can access it.
"""

gr.ChatInterface(chat, type="messages").launch(share=True)

## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [ ]:
"""
This code defines a Pydantic model named 'Evaluation' to structure evaluation data.

The model includes:
- is_acceptable (bool): Indicates whether the submission meets the criteria.
- feedback (str): Provides written feedback or suggestions for improvement.

Pydantic ensures type validation and data consistency.
"""

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [ ]:
"""
This code builds a system prompt for an AI evaluator agent.

The evaluator's role is to assess the quality of an Agent's response in a simulated conversation,
where the Agent is acting as {name} on their personal/professional website.

The evaluator receives context including {name}'s summary and LinkedIn profile,
and is instructed to determine whether the Agent's latest reply is acceptable,
while providing constructive feedback.
"""

evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
"""
This function generates a user prompt for the evaluator agent.

It organizes the full conversation context by including:
- the full chat history,
- the most recent user message,
- and the most recent agent reply.

The final prompt instructs the evaluator to assess the quality of the agent’s response,
and return both an acceptability judgment and constructive feedback.
"""

def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
"""
This script tests whether the Google Generative AI API key is working correctly.

- It loads the API key from a .env file using `dotenv`.
- Initializes a genai.Client with the loaded key.
- Attempts to generate a simple response using the "gemini-2.0-flash" model.
- Prints confirmation if the key is valid, or shows an error message if the request fails.
"""

from dotenv import load_dotenv
import os
from google import genai

load_dotenv()

client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))

try:
    # Use the correct method for genai.Client
    test_response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents="Hello"
    )
    print("✅ API key is working!")
    print(f"Response: {test_response.text}")
except Exception as e:
    print(f"❌ API key test failed: {e}")



In [ ]:
"""
This line initializes an OpenAI-compatible client for accessing Google's Generative Language API.

- `api_key` is retrieved from environment variables.
- `base_url` points to Google's OpenAI-compatible endpoint.

This setup allows you to use OpenAI-style syntax to interact with Google's Gemini models.
"""

gemini = OpenAI(
    api_key=os.environ.get("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
"""
This function sends a structured evaluation request to the Gemini API and returns a parsed `Evaluation` object.

- It constructs the message list using:
  - a system prompt defining the evaluator's role and context
  - a user prompt containing the conversation history, user message, and agent reply

- It uses Gemini's OpenAI-compatible API to process the evaluation request,
  specifying `response_format=Evaluation` to get a structured response.

- The function returns the parsed evaluation result (acceptability and feedback).
"""

def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
"""
This code sends a test question to the AI agent and evaluates its response.

1. It builds a message list including:
   - the system prompt that defines the agent’s role
   - a user question: "do you hold a patent?"

2. The message list is sent to OpenAI's GPT-4o-mini model to generate a response.

3. The reply is extracted from the API response.

4. The `evaluate()` function is then called with:
   - the agent’s reply
   - the original user message
   - and just the system prompt as history (no prior user/agent exchange)

This allows automated evaluation of how well the agent answers the question.
"""

messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content
reply
evaluate(reply, "do you hold a patent?", messages[:1])

In [ ]:
"""
This function re-generates a response after a previous reply was rejected during evaluation.

It:
1. Appends rejection feedback to the original system prompt to inform the agent of:
   - its previous answer,
   - and the reason it was rejected.

2. Reconstructs the full message list including:
   - the updated system prompt,
   - the prior conversation history,
   - and the original user message.

3. Sends the updated prompt to OpenAI's GPT-4o-mini model.

4. Returns a revised response from the model that ideally addresses the feedback.
"""
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
"""
This function handles a chat interaction with conditional behavior and automatic quality control.

Steps:
1. If the user's message contains the word "patent", the agent is instructed to respond entirely in Pig Latin by appending an instruction to the system prompt.
2. Constructs the full message history including the updated system prompt, prior conversation, and the new user message.
3. Sends the request to OpenAI's GPT-4o-mini model and receives a reply.
4. Evaluates the reply using a separate evaluator agent to determine if the response meets quality standards.
5. If the evaluation passes, the reply is returned.
6. If the evaluation fails, the function logs the feedback and calls `rerun()` to generate a corrected reply based on the feedback.
"""

def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [1]:
"""
This launches a Gradio chat interface using the `chat` function.

- `type="messages"` enables multi-turn chat with message bubbles.
- `share=True` generates a public link so others can interact with the app.
"""
gr.ChatInterface(chat, type="messages").launch(share=True)

'\nThis launches a Gradio chat interface using the `chat` function.\n\n- `type="messages"` enables multi-turn chat with message bubbles.\n- `share=True` generates a public link so others can interact with the app.\n'